In [1]:
# load packages
import os
import sys
import re
import random
import numpy as np
import pandas as pd
import tensorflow as tf
tf.__version__

'1.12.0'

## Prepare Data

In [2]:
data = pd.read_csv("data/iris-dataset.csv") # load data
type(data)

pandas.core.frame.DataFrame

In [3]:
data.shape

(150, 6)

In [4]:
data.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
target_values = data.Species.unique()
target_values

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [6]:
# create mapping string -> numeric value
target_values_mapping = dict()
i = 0
for tar in target_values:
    if tar not in target_values_mapping:
        target_values_mapping[tar] = i
        i += 1
    else:
        continue

In [7]:
target_values_mapping

{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

In [8]:
# convert string value to numeric values
data.Species = data.Species.apply(lambda x: target_values_mapping[x])
data.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0


## Selecting Only 2 Classes -> Converting Problem Into Binary Classification

In [9]:
data = data[data.Species <= 1]
data.shape

(100, 6)

In [10]:
data.Species.unique()

array([0, 1])

In [11]:
features = np.array(data.iloc[:, 1:-1]) # load features into a numpy array
features.shape

(100, 4)

In [12]:
labels = np.array(data.iloc[:, [-1]]) # load target values into a numpy array
labels.shape

(100, 1)

In [13]:
features[0], labels[0]

(array([5.1, 3.5, 1.4, 0.2]), array([0]))

In [14]:
features[1], labels[1]

(array([4.9, 3. , 1.4, 0.2]), array([0]))

In [15]:
# split data into train and validate
val_split_size = 0.2
random_indices = np.random.rand(len(features)) <= val_split_size

# split data
train_x = features[~random_indices]
val_x = features[random_indices]
train_y = labels[~random_indices]
val_y = labels[random_indices]

In [16]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape

((81, 4), (81, 1), (19, 4), (19, 1))

## Vectorized Logistic Regression Model

In [17]:
# placeholders
x = tf.placeholder(dtype=tf.float32, shape=(None, features.shape[1]), name="feature_x")
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name="target_y")

In [18]:
# define weights and bias
w = tf.Variable(tf.random_normal(shape=(1, features.shape[1]), dtype=tf.float32, mean=0.0, stddev=0.1))
b = tf.Variable(tf.zeros(shape=(1, 1), dtype=tf.float32))

In [19]:
logistic_model = tf.sigmoid(tf.add(tf.matmul(x, tf.transpose(w)), b))

In [20]:
# loss to minimize
loss_function = tf.reduce_mean(-(y*tf.log(logistic_model) + (1 - y)*tf.log(1 - logistic_model))) # cross entropy

## Using Inbuilt Optimizer

In [21]:
trainer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss_function)

In [22]:
# train the model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("[Training...]")
    for epoch in range(10):
        sess.run(trainer, feed_dict={x:train_x, y:train_y})
        train_loss = sess.run(loss_function, feed_dict={x:train_x, y:train_y})
        print("Step:", epoch, "\tTrain Loss:", train_loss, end="\t")
        val_loss = sess.run(loss_function, feed_dict={x:val_x, y:val_y})
        print("Val Loss:", val_loss)
    print("Final Weights:", sess.run(w))
    predicted_y = sess.run(logistic_model, feed_dict={x:val_x, y:val_y})

[Training...]
Step: 0 	Train Loss: 0.7442417	Val Loss: 0.6936076
Step: 1 	Train Loss: 0.6912273	Val Loss: 0.74718904
Step: 2 	Train Loss: 0.63678056	Val Loss: 0.7218525
Step: 3 	Train Loss: 0.5473272	Val Loss: 0.61203665
Step: 4 	Train Loss: 0.45977688	Val Loss: 0.48521116
Step: 5 	Train Loss: 0.40125728	Val Loss: 0.39041895
Step: 6 	Train Loss: 0.3642385	Val Loss: 0.33146533
Step: 7 	Train Loss: 0.3295988	Val Loss: 0.29087383
Step: 8 	Train Loss: 0.2899259	Val Loss: 0.25728118
Step: 9 	Train Loss: 0.24904405	Val Loss: 0.22902097
Final Weights: [[-0.03242555 -0.40932512  0.51922035  0.98098624]]


### > Prediction Analysis

In [23]:
# clip the prediction on threshold of 0.5 to get final prediction
final_prediction = list()
for pred in predicted_y:
    if pred > 0.5:
        final_prediction.append(1)
    else:
        final_prediction.append(0)
final_prediction = np.array(final_prediction).reshape((len(final_prediction), 1))
final_prediction.shape, val_y.shape

((19, 1), (19, 1))

In [24]:
# find accuracy
correct_prediciton = 0
for i in range(val_y.shape[0]):
    if final_prediction[i] == val_y[i]:
        correct_prediciton += 1
    else:
        continue
"Accuracy = ", correct_prediciton / val_y.shape[0] * 100

('Accuracy = ', 100.0)

## Use Your Own Gradient Descent

In [25]:
# compute gradients
grad_w, grad_b = tf.gradients(xs=[w, b], ys=loss_function)
# update
new_w = w.assign(w - 0.01*grad_w)
new_b = b.assign(b - 0.01*grad_b)

In [26]:
# train the model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("[Training...]")
    for epoch in range(10):
        _, _, train_loss = sess.run([new_w, new_b, loss_function], feed_dict={x:train_x, y:train_y})
        print("Step:", epoch, "\tTrain Loss:", train_loss, end="\t")
        val_loss = sess.run(loss_function, feed_dict={x:val_x, y:val_y})
        print("Val Loss:", val_loss)
    print("Final Weights:", sess.run(w))
    predicted_y = sess.run(logistic_model, feed_dict={x:val_x, y:val_y})

[Training...]
Step: 0 	Train Loss: 0.63022244	Val Loss: 0.5537298
Step: 1 	Train Loss: 0.62141633	Val Loss: 0.5531254
Step: 2 	Train Loss: 0.61371714	Val Loss: 0.5526054
Step: 3 	Train Loss: 0.6069011	Val Loss: 0.552076
Step: 4 	Train Loss: 0.60078734	Val Loss: 0.5514722
Step: 5 	Train Loss: 0.5952317	Val Loss: 0.55075145
Step: 6 	Train Loss: 0.59011966	Val Loss: 0.54988813
Step: 7 	Train Loss: 0.5853607	Val Loss: 0.54886925
Step: 8 	Train Loss: 0.5808837	Val Loss: 0.547691
Step: 9 	Train Loss: 0.57663304	Val Loss: 0.5463559
Final Weights: [[ 0.02471445 -0.10241345  0.082486    0.22958131]]


### > Prediction Analysis

In [27]:
# clip the prediction on threshold of 0.5 to get final prediction
final_prediction = list()
for pred in predicted_y:
    if pred > 0.5:
        final_prediction.append(1)
    else:
        final_prediction.append(0)
final_prediction = np.array(final_prediction).reshape((len(final_prediction), 1))
final_prediction.shape, val_y.shape

((19, 1), (19, 1))

In [28]:
# find accuracy
correct_prediciton = 0
for i in range(val_y.shape[0]):
    if final_prediction[i] == val_y[i]:
        correct_prediciton += 1
    else:
        continue
"Accuracy = ", correct_prediciton / val_y.shape[0] * 100

('Accuracy = ', 100.0)

Perfect Score!